In [3]:
import requests

In [1]:
url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

payload ={
"bld" : "dbms/MDC/STAT/standard/MDCSTAT01901",
"locale": "ko_KR",
"mktId": "ALL",
"share": "1",
"csvxls_isNo" : "false",
}

head = {"User-Agent" :
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}

In [4]:
r = requests.post(url, payload, headers=head)

In [21]:
# r.text

## check this
# r.json()['OutBlock_1']

In [6]:
import pandas as pd

df = pd.DataFrame(r.json()['OutBlock_1'])
# df

### 무슨 요일에 상장을 했을 지 알아보는 전처리 1

In [33]:
from datetime import datetime

date_format = "%Y/%m/%d" 
weekday_values = []  

days = {
    0: 'Mon', 
    1: 'Tue', 
    2: 'Wed', 
    3: 'Thu', 
    4: 'Fri', 
    5: 'Sat', 
    6: 'Sun'
    }

for i in df['LIST_DD']:
    date_obj = datetime.strptime(i, date_format)

    weekday = days[date_obj.weekday()]
    weekday_values.append(weekday)

df['LIST_DD'] = weekday_values 

In [34]:
df['LIST_DD'].value_counts()

LIST_DD
Fri    667
Thu    573
Tue    557
Wed    490
Mon    383
Sat     78
Name: count, dtype: int64

### 무슨 요일에 상장을 했을 지 알아보는 전처리 2

In [72]:
df['LIST_DD'] = pd.to_datetime(df['LIST_DD'])

In [73]:
master = dict(zip(range(0,7),"월화수목금토일"))

In [74]:
df['LIST_DD'].apply(lambda x : master[x.weekday()]).value_counts()

LIST_DD
금    667
목    573
화    557
수    490
월    383
토     78
Name: count, dtype: int64

In [75]:
df[df['LIST_DD'].apply(lambda x : master[x.weekday()] == "토")].sort_values("LIST_DD").tail(5)

,ISU_CD,ISU_SRT_CD,ISU_NM,ISU_ABBRV,ISU_ENG_NM,LIST_DD,MKT_TP_NM,SECUGRP_NM,SECT_TP_NM,KIND_STKCERT_TP_NM,PARVAL,LIST_SHRS
994,KR7015750003,015750,성우하이텍,성우하이텍,"SUNGWOO HITECH CO., LTD",1995-11-04,KOSDAQ,주권,우량기업부,보통주,500,"80,000,000"
1200,KR7025980004,025980,아난티,아난티,Ananti Inc.,1996-01-06,KOSDAQ,주권,우량기업부,보통주,100,"87,629,522"
2379,KR7027710003,027710,팜스토리,팜스토리,"FarmStory CO.,LTD.",1996-07-06,KOSDAQ,주권,우량기업부,보통주,500,"111,416,600"
1435,KR7028300002,028300,에이치엘비,HLB,HLB INC.,1996-07-27,KOSDAQ,주권,중견기업부,보통주,500,"128,929,651"
910,KR7032280000,032280,삼일,삼일,SamIl,1997-03-15,KOSDAQ,주권,중견기업부,보통주,"1,000","16,213,590"


In [76]:
from datetime import date, datetime, timedelta
df['현재까지'] = df['LIST_DD'].apply(lambda x : (datetime.now()-x).days)

In [77]:
# df.sort_values(by="현재까지")
df[df['현재까지']<100].sort_values(by="현재까지",ascending=False).head()

,ISU_CD,ISU_SRT_CD,ISU_NM,ISU_ABBRV,ISU_ENG_NM,LIST_DD,MKT_TP_NM,SECUGRP_NM,SECT_TP_NM,KIND_STKCERT_TP_NM,PARVAL,LIST_SHRS,현재까지
2311,KR7417790003,417790,트루엔,트루엔,"TRUEN Co., Ltd",2023-05-17,KOSDAQ,주권,벤처기업부,보통주,100,"10,999,650",99
2260,KR7446840001,446840,키움제8호기업인수목적,키움제8호스팩,Kiwoom No.8 Special Purpose Acquisition Company,2023-05-17,KOSDAQ,주권,SPAC(소속부없음),보통주,100,"5,880,000",99
2229,KR7456190008,456190,큐라켐,큐라켐,CURACHEM,2023-05-18,KONEX,주권,일반기업부,보통주,500,"1,754,286",98
1181,KR7340810001,340810,씨유박스,씨유박스,"CUBOX Co., Ltd",2023-05-19,KOSDAQ,주권,기술성장기업부,보통주,500,"10,347,861",97
685,KR7434480000,434480,모니터랩,모니터랩,"MONITORAPP CO., LTD.",2023-05-19,KOSDAQ,주권,기술성장기업부,보통주,100,"12,083,500",97


In [8]:
# KOSDAQ, KOSPI 에만 상장된 주식 데이터 가져오기

target = df.loc[df.MKT_TP_NM.apply(lambda x : (x == 'KOSPI') or (x =='KOSDAQ'))].copy()
target.shape
##  위와 같은 결과 값.
# target = df[df['MKT_TP_NM'].isin(["KOSPI","KOSDAQ"])].copy()
# target['ISU_SRT_CD']

(2570, 12)

In [25]:
target.sort_values('LIST_DD',ascending=False).head()

,ISU_CD,ISU_SRT_CD,ISU_NM,ISU_ABBRV,ISU_ENG_NM,LIST_DD,MKT_TP_NM,SECUGRP_NM,SECT_TP_NM,KIND_STKCERT_TP_NM,PARVAL,LIST_SHRS
1094,KR7418250007,418250,시큐레터,시큐레터,"SecuLetter Co., Ltd.",2023-08-24,KOSDAQ,주권,기술성장기업부,보통주,500,"7,911,521"
1071,KR7424960003,424960,스마트레이더시스템,스마트레이더시스템,"Smart Radar System, Inc.",2023-08-22,KOSDAQ,주권,기술성장기업부,보통주,500,"14,836,340"
293,KR7092790005,092790,넥스틸보통주,넥스틸,NEXTEEL,2023-08-21,KOSPI,주권,,보통주,500,"26,002,000"
844,KR7210120002,210120,빅토리콘텐츠,빅텐츠,"Victory Contents Co.,Ltd.",2023-08-17,KOSDAQ,주권,중견기업부,보통주,500,"3,144,610"
1383,KR7457940005,457940,에스케이증권제10호기업인수목적,에스케이증권제10호스팩,SK Securities No.10 Special Purpose Acquisitio...,2023-08-11,KOSDAQ,주권,SPAC(소속부없음),보통주,100,"3,310,000"


## 네이버 증권 사이트에서 시세 가져오기

In [12]:
import requests

In [38]:
sise_url = "https://api.finance.naver.com/siseJson.naver?symbol=004830&requestType=2&count=1000&startTime=20230823&timeframe=day"

In [39]:
r2 = requests.post(sise_url)
r2

<Response [200]>

In [85]:
eval(r2.text.replace("\n","").strip())[0]

tmp = eval(r2.text.replace("\n","").strip())

In [47]:
# pd.DataFrame(tmp[1:], columns=tmp[0])

# stock 폴더에 종목코드 `.csv` 형태로 저장 (KOSPI, KOSDAQ)

In [40]:
sise_url2 = "https://api.finance.naver.com/siseJson.naver?symbol={}&requestType=1&startTime=2020101&endTime=20230823&timeframe=day"



In [55]:
import tqdm
## 앞으로 utf-8-sig 이것으로 인코딩하자

for i in tqdm.tqdm(target['ISU_SRT_CD']):
    r3 = requests.post(sise_url2.format(i))
    tmp = eval(r3.text.replace("\n","").strip())
    temp_df = pd.DataFrame(tmp[1:], columns=tmp[0])
    temp_df['StockCode'] = i
     
    temp_df.to_csv("C:\workspace\Bootcamp_basic_DataEngineering\stock\{}.csv".format(i),encoding="utf-8-sig")
    

100%|██████████| 2570/2570 [02:41<00:00, 15.88it/s]


In [101]:
help(pd.DataFrame.to_csv)

Help on function to_csv in module pandas.core.generic:

to_csv(self, path_or_buf: 'FilePath | WriteBuffer[bytes] | WriteBuffer[str] | None' = None, sep: 'str' = ',', na_rep: 'str' = '', float_format: 'str | Callable | None' = None, columns: 'Sequence[Hashable] | None' = None, header: 'bool_t | list[str]' = True, index: 'bool_t' = True, index_label: 'IndexLabel | None' = None, mode: 'str' = 'w', encoding: 'str | None' = None, compression: 'CompressionOptions' = 'infer', quoting: 'int | None' = None, quotechar: 'str' = '"', lineterminator: 'str | None' = None, chunksize: 'int | None' = None, date_format: 'str | None' = None, doublequote: 'bool_t' = True, escapechar: 'str | None' = None, decimal: 'str' = '.', errors: 'str' = 'strict', storage_options: 'StorageOptions' = None) -> 'str | None'
    Write object to a comma-separated values (csv) file.
    
    Parameters
    ----------
    path_or_buf : str, path object, file-like object, or None, default None
        String, path object (imp

In [54]:
pd.read_csv("C:\workspace\Bootcamp_basic_DataEngineering\stock\\001045.csv").head()

,Unnamed: 0,날짜,시가,고가,저가,종가,거래량,외국인소진율,StockCode
0,0,20201012,47500,47900,47200,47800,5710,8.29,1045
1,1,20201013,47800,48100,47550,48100,6059,8.31,1045
2,2,20201014,48100,48100,47400,47700,7670,8.33,1045
3,3,20201015,48250,48250,47200,47700,3207,8.33,1045
4,4,20201016,47700,47700,46100,47450,5199,8.34,1045


### 위와 같지만 다른 코드

In [13]:
import os
import csv
from tqdm import tqdm


In [15]:
os.path

<module 'ntpath' from 'C:\\Users\\Playdata\\anaconda3\\lib\\ntpath.py'>

In [ ]:

## 주소 안에 stock 폴더 있는 지 여부 확인
### 이후 없다면 생성까지.
if os.path.isdir("./stock")  == False:
    os.mkdir("./stock")

In [ ]:

### csv 패키지를 활용해서 작성.
for code in tqdm(target['ISU_SRT_CD']):
    r2 = requests.post(sise_url.format(code))
    data_list = eval(r2.text.replace("\n", "").strip())
    with open('./stock/{}.csv'.format(code), 'w', encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for x in data_list:
            writer.writerow(x)

# 다운 받은 `csv` 파일들 `DB`에 저장.

In [1]:
import pandas as pd
import pymysql

In [56]:
con = pymysql.connect(host='localhost', user='root', password='8489',
                db='encore', charset = 'utf8')

In [59]:
cur = con.cursor()
cur.execute("show tables")

7

In [58]:
cr_table ="""CREATE TABLE stocks(
    StockCode VARCHAR(255),
    Date DATE,
    Open INT(11),
    High INT(11),
    Low INT(11),
    Close INT(11),
    Volume INT(11),
    foreignExRate FLOAT(7,2)
);"""

In [60]:
cur.execute(cr_table)

0

In [61]:
sql = "INSERT into stocks VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"

In [62]:
csv_file = pd.read_csv("C:\workspace\Bootcamp_basic_DataEngineering\stock\\001045.csv")